# Using sklearn and pytorch

* Demonstrate how someone can directly use sklearn learners or pytorch in CapyMOA.
* Ideally, one should be free to use other learners

**Accessing the input data x()**

* Accessing the input data as a double array from an ```Instance``` through function ```x()```
* Instances are represented internally as MOA Instances. 

**notebook last updated on 08/12/2023**

## 0. Reading data and accessing x()

In [1]:
from capymoa.stream import stream_from_file

DATA_PATH = "../data/"

## Opening a file as a stream
elec_stream = stream_from_file(path_to_csv_or_arff=DATA_PATH+"electricity.csv")

elec_stream.restart()
i = 0
while elec_stream.has_more_instances():
    instance = elec_stream.next_instance()
    if i < 20: # prevent printing all the instances
        print(f'x: {instance.x}, y: {instance.y_label}')
    i+=1

capymoa_root: /home/antonlee/github.com/tachyonicClock/MOABridge/src/capymoa
MOA jar path location (config.ini): /home/antonlee/github.com/tachyonicClock/MOABridge/src/capymoa/jar/moa.jar
JVM Location (system): 
JAVA_HOME: /usr/lib/jvm/java-17-openjdk
JVM args: ['-Xmx8g', '-Xss10M']


Sucessfully started the JVM and added MOA jar to the class path


x: [0.       0.056443 0.439155 0.003467 0.422915 0.414912], y: 1
x: [0.021277 0.051699 0.415055 0.003467 0.422915 0.414912], y: 1
x: [0.042553 0.051489 0.385004 0.003467 0.422915 0.414912], y: 1
x: [0.06383  0.045485 0.314639 0.003467 0.422915 0.414912], y: 1
x: [0.085106 0.042482 0.251116 0.003467 0.422915 0.414912], y: 0
x: [0.106383 0.041161 0.207528 0.003467 0.422915 0.414912], y: 0
x: [0.12766  0.041161 0.171824 0.003467 0.422915 0.414912], y: 0
x: [0.148936 0.041161 0.152782 0.003467 0.422915 0.414912], y: 0
x: [0.170213 0.041161 0.13493  0.003467 0.422915 0.414912], y: 0
x: [0.191489 0.041161 0.140583 0.003467 0.422915 0.414912], y: 0
x: [0.212766 0.044374 0.168997 0.003467 0.422915 0.414912], y: 1
x: [0.234043 0.049868 0.212437 0.003467 0.422915 0.414912], y: 1
x: [0.255319 0.051489 0.298721 0.003467 0.422915 0.414912], y: 1
x: [0.276596 0.042482 0.39036  0.003467 0.422915 0.414912], y: 0
x: [0.297872 0.040861 0.402261 0.003467 0.422915 0.414912], y: 0
x: [0.319149 0.040711 0.4

In [2]:
# Getting some extra information about the instance through the MOA representation. 
moa_instance = instance.java_instance.getData()

for i in range(0, moa_instance.numInputAttributes()):
    print(moa_instance.attribute(i))
    print(moa_instance.value(i))

@attribute period numeric
1.0
@attribute nswprice numeric
0.050679
@attribute nswdemand numeric
0.288753
@attribute vicprice numeric
0.003542
@attribute vicdemand numeric
0.355256
@attribute transfer numeric
0.23114


## 1. Using scikit-learn

* Example showing how a model from scikit-learn can be used with our ```Instance``` representation

In [3]:

from sklearn import linear_model
from capymoa.evaluation import ClassificationEvaluator
from capymoa.datasets import ElectricityTiny

# Creating a stream. Using the tiny version of the electricity dataset to speed
# up the process
elec_stream = ElectricityTiny()

# Creating a learner
sklearn_SGD = linear_model.SGDClassifier()

# Creating the evaluator
ob_evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

# elec_stream.schema.get_label_indexes() --> the class labels

# Counter for partial fits
partial_fit_count = 0
while elec_stream.has_more_instances():
    instance = elec_stream.next_instance()

    prediction = -1
    if partial_fit_count > 0: # scikit-learn does not allows invoking predict in a model that was not fit before
        prediction = sklearn_SGD.predict([instance.x])[0]
    ob_evaluator.update(instance.y_label, instance.schema.get_value_for_index(prediction))
    sklearn_SGD.partial_fit([instance.x], [instance.y_index], classes=elec_stream.schema.get_label_indexes())
    partial_fit_count += 1

ob_evaluator.accuracy()

84.7

### 1.1 Example using a MOA learner



In [4]:
from moa.classifiers.trees import HoeffdingAdaptiveTree
from capymoa.evaluation import ClassificationEvaluator
from capymoa.learner import MOAClassifier

## Opening a file as a stream
elec_stream = ElectricityTiny()

# Creating a learner
moa_HAT = MOAClassifier(schema=elec_stream.get_schema(), moa_learner=HoeffdingAdaptiveTree())

# Creating the evaluator
hat_evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

while elec_stream.has_more_instances():
    instance = elec_stream.next_instance()

    prediction = moa_HAT.predict(instance)
    hat_evaluator.update(instance.y_label, prediction)
    moa_HAT.train(instance)
    partial_fit_count += 1

hat_evaluator.accuracy()

82.75

### 1.2 Using SKClassifier


In [5]:
from sklearn import linear_model
from capymoa.learner import SKClassifier
from capymoa.evaluation import ClassificationEvaluator

## Opening a file as a stream
elec_stream = ElectricityTiny()

# Creating a learner
sklearn_SGD = SKClassifier(schema=elec_stream.get_schema(), sklearner=linear_model.SGDClassifier())

# Creating the evaluator
sklearn_SGD_evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

while elec_stream.has_more_instances():
    instance = elec_stream.next_instance()

    prediction = sklearn_SGD.predict(instance)
    sklearn_SGD_evaluator.update(instance.y_label, instance.schema.get_value_for_index(prediction))
    sklearn_SGD.train(instance)

sklearn_SGD_evaluator.accuracy()

84.7

### 1.3 Using prequential evaluation + SKClassifier

In [6]:
from capymoa.evaluation import prequential_evaluation

## Opening a file as a stream
elec_stream = stream_from_file(path_to_csv_or_arff=DATA_PATH+"electricity.csv")

# Creating a learner
sklearn_SGD = SKClassifier(schema=elec_stream.get_schema(), sklearner=linear_model.SGDClassifier())

results_sklearn_SGD = prequential_evaluation(stream=elec_stream, learner=sklearn_SGD, window_size=4500)

results_sklearn_SGD['cumulative'].accuracy()

57.54546257062147

## 2. Using PyTorch
* Example showing how a simple Pytorch model can be used with our ```Instance``` representation and MOA evaluator
* Uses CPU device
* Model is initialized after receiving the first instance

#### Set random seeds

In [7]:
import random
random_seed=1
random.seed(random_seed)

#### Define network structure

In [8]:
import torch
from torch import nn

torch.manual_seed(random_seed)
torch.use_deterministic_algorithms(True)

# Get cpu device for training.
device = ("cpu")
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=0, number_of_classes=0):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, number_of_classes)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = None
optimizer = None
loss_fn = nn.CrossEntropyLoss()

Using cpu device


#### Using instance loop

In [9]:
from capymoa.evaluation import ClassificationEvaluator

## Opening a file again to start from the beginning
elec_stream = ElectricityTiny()

# Creating the evaluator
evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

i = 0
while elec_stream.has_more_instances():
    i += 1
    instance = elec_stream.next_instance()
    if model is None:
        moa_instance = instance.java_instance.getData()
        # initialize the model and send it to the device
        model = NeuralNetwork(input_size=elec_stream.get_schema().get_num_attributes(), 
                              number_of_classes=elec_stream.get_schema().get_num_classes()).to(device)
        # set the optimizer
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
        print(model)
    
    X = torch.tensor(instance.x, dtype=torch.float32)
    y = torch.tensor(instance.y_index, dtype=torch.long)
    # set the device and add a dimension to the tensor
    X, y = torch.unsqueeze(X.to(device), 0), torch.unsqueeze(y.to(device),0) 
    
    # turn off gradient collection for test
    with torch.no_grad():
        pred = model(X)
        prediction = instance.schema.get_value_for_index(torch.argmax(pred))

    # update evaluator with predicted class
    evaluator.update(instance.y_label, prediction)
  
    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if i % 500 == 0:
        print(f'Accuracy at {i} : {evaluator.accuracy()}')
    
print(f'Accuracy at {i} : {evaluator.accuracy()}')

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=6, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


Accuracy at 500 : 50.4


Accuracy at 1000 : 55.2


Accuracy at 1500 : 61.199999999999996


Accuracy at 2000 : 61.1
Accuracy at 2000 : 61.1


### 2.1 PyTorchClassifier

In [10]:
from capymoa.learner import Classifier
import numpy as np

class PyTorchClassifier(Classifier):
    def __init__(self, schema=None, random_seed=1, nn_model: nn.Module = None, optimizer=None, loss_fn=nn.CrossEntropyLoss(), device=("cpu"), lr=1e-3):
        super().__init__(schema, random_seed)
        self.model = None
        self.optimizer = None
        self.loss_fn = loss_fn
        self.lr = lr
        self.device = device
        
        torch.manual_seed(random_seed)
        
        if nn_model is None:
            self.set_model(None)
        else:
            self.model = nn_model.to(device)
        if optimizer is None:
            if self.model is not None:
                self.optimizer = torch.optim.SGD(self.model.parameters(), lr=lr)
        else:
            self.optimizer = optimizer
        
    def __str__(self):
        return str(self.model)

    def CLI_help(self):
        return str('schema=None, random_seed=1, nn_model: nn.Module = None, optimizer=None, loss_fn=nn.CrossEntropyLoss(), device=("cpu"), lr=1e-3')

    def set_model(self, instance):
        if self.schema is None:
            moa_instance = instance.java_instance.getData()
            self.model = NeuralNetwork(input_size=moa_instance.get_num_attributes(), number_of_classes=moa_instance.get_num_classes()).to(self.device)
        elif instance is not None:
            self.model = NeuralNetwork(input_size=self.schema.get_num_attributes(), number_of_classes=self.schema.get_num_classes()).to(self.device)
            
    def train(self, instance):
        if self.model is None:
            self.set_model(instance)
    
        X = torch.tensor(instance.x, dtype=torch.float32)
        y = torch.tensor(instance.y_index, dtype=torch.long)
        # set the device and add a dimension to the tensor
        X, y = torch.unsqueeze(X.to(self.device), 0), torch.unsqueeze(y.to(self.device),0)

        # Compute prediction error
        pred = self.model(X)
        loss = self.loss_fn(pred, y)
    
        # Backpropagation
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()

    def predict(self, instance):
        return instance.schema.get_value_for_index(np.argmax(self.predict_proba(instance)))

    def predict_proba(self, instance):
        if self.model is None:
            self.set_model(instance)
        X = torch.unsqueeze(torch.tensor(instance.x, dtype=torch.float32).to(self.device), 0)
        # turn off gradient collection
        with torch.no_grad():
            pred = np.asarray(self.model(X).numpy(), dtype=np.double)
        return pred


#### 2.1.1 Example using PyTorchClassifier + the instance loop

In [11]:
from capymoa.evaluation import ClassificationEvaluator

## Opening a file again to start from the beginning
elec_stream = ElectricityTiny()

# Creating the evaluator
evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

# Creating a learner
simple_pyTorch_classifier = PyTorchClassifier(
    schema=elec_stream.get_schema(), 
    nn_model=NeuralNetwork(input_size=elec_stream.get_schema().get_num_attributes(), number_of_classes=elec_stream.get_schema().get_num_classes()).to(device)
)

while elec_stream.has_more_instances():
    instance = elec_stream.next_instance()

    prediction = simple_pyTorch_classifier.predict(instance)
    evaluator.update(instance.y_label, prediction)
    simple_pyTorch_classifier.train(instance)

evaluator.accuracy()

62.849999999999994

#### 2.1.2 Example using PyTorchClassifier and prequential_evaluation

In [12]:
from capymoa.evaluation import prequential_evaluation

## Opening a file as a stream
elec_stream = ElectricityTiny()

# Creating a learner
simple_pyTorch_classifier = PyTorchClassifier(
    schema=elec_stream.get_schema(), 
    nn_model=NeuralNetwork(input_size=elec_stream.get_schema().get_num_attributes(), number_of_classes=elec_stream.get_schema().get_num_classes()).to(device)
)

evaluator = prequential_evaluation(stream=elec_stream, learner=simple_pyTorch_classifier, window_size=4500, optimise=False)

evaluator['cumulative'].accuracy()

61.1

### 2.2 How to use TensorBoard with PyTorch

Install TensorBoard through the command line to visualize data you logged

```sh
pip install tensorboard
```
Clear any logs from previous runs

```sh
rm -rf ./runs
```

**TODO: create another notebook (for visualizations) and move this section**

In [13]:
!pip install tensorboard

#### Create a SummaryWriter instance.

In [14]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

#### Example using PyTorchClassifier + the instance loop + TensorBoard

In [15]:
from capymoa.evaluation import ClassificationEvaluator

## Opening a file again to start from the beginning
elec_stream = ElectricityTiny()

# Creating the evaluator
evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

# Creating a learner
simple_pyTorch_classifier = PyTorchClassifier(
    schema=elec_stream.get_schema(), 
    nn_model=NeuralNetwork(input_size=elec_stream.get_schema().get_num_attributes(), number_of_classes=elec_stream.get_schema().get_num_classes()).to(device)
)

i = 0
while elec_stream.has_more_instances():
    i += 1
    instance = elec_stream.next_instance()

    prediction = simple_pyTorch_classifier.predict(instance)
    evaluator.update(instance.y_label, prediction)
    simple_pyTorch_classifier.train(instance)
    
    if i % 1000 == 0:
        writer.add_scalar("accuracy", evaluator.accuracy(), i)

writer.add_scalar("accuracy", evaluator.accuracy(), i)
writer.flush()

Call flush() method to make sure that all pending events have been written to disk.

See torch.utils.tensorboard tutorials to find more TensorBoard visualization types you can log.

In [16]:
# If you do not need the summary writer anymore, call close() method.
writer.close()

```
# This is formatted as code
```

#### Run TensorBoard
Now, start TensorBoard, specifying the root log directory you used above. 
Argument ``logdir`` points to directory where TensorBoard will look to find 
event files that it can display. TensorBoard will recursively walk 
the directory structure rooted at ``logdir``, looking for ``.*tfevents.*`` files.

```sh
tensorboard --logdir=runs
```
Go to the URL it provides

This dashboard shows how the accuracy change with time. 
You can use it to also track training speed, learning rate, and other 
scalar values.

### 2.3 How to use pytorch dataset with a CapyMOA classifier

In [ ]:
from capymoa.evaluation import ClassificationEvaluator
from capymoa.learner.classifier.classifiers import OnlineBagging
from capymoa.stream import PytorchStream

from torchvision import datasets
from torchvision.transforms import ToTensor

pytorchDtaset = datasets.FashionMNIST(
            root="data",
            train=True,
            download=True,
            transform=ToTensor()
        )
pytorchc_stream = PytorchStream(dataset=pytorchDtaset)

# Creating a learner
ob_learner = OnlineBagging(schema=pytorchc_stream.get_schema(), ensemble_size=5)

# Creating the evaluator
ob_evaluator = ClassificationEvaluator(schema=pytorchc_stream.get_schema())

i = 0
while pytorchc_stream.has_more_instances() and i < 1000:
    instance = pytorchc_stream.next_instance()
    
    prediction = ob_learner.predict(instance)
    ob_evaluator.update(instance.y_label, prediction)
    ob_learner.train(instance)
    i += 1 

ob_evaluator.accuracy()

## 4. Preprocessing using MOA

* Includes an example of how preprocessing (from MOA) can be used, **maybe this example could be moved elsewhere**
* ```x()``` is read-only as of now, so one cannot preprocess instances
* **TODO**: Allow modifying ```x()``` so that python-based preprocessing can be used. 

### 4.1 Running onlineBagging without any preprocessing

In [17]:
## Test-then-train loop
from capymoa.learner.classifier import OnlineBagging
from capymoa.evaluation import ClassificationEvaluator

## Opening a file as a stream
elec_stream = stream_from_file(path_to_csv_or_arff=DATA_PATH+"electricity.csv")

# Creating a learner
ob_learner = OnlineBagging(schema=elec_stream.get_schema(), ensemble_size=5)

# Creating the evaluator
ob_evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

while elec_stream.has_more_instances():
    instance = elec_stream.next_instance()
    
    prediction = ob_learner.predict(instance)
    ob_evaluator.update(instance.y_label, prediction)
    ob_learner.train(instance)

ob_evaluator.accuracy()

79.05190677966102

### 4.2 Online Bagging using the preprocessing method from MOA
* The API is still a bit rough

In [18]:
# shows the creation string, the __class__ is needed as a parameter to the function is the class used. 
elec_stream.CLI

'RandomTreeGenerator '

In [19]:
# Show the number of attributes (including the output)
elec_stream.get_schema().num_attributes_including_output

7

In [20]:
from capymoa.stream import Stream
from moa.streams.filters import StandardisationFilter, NormalisationFilter
from moa.streams import FilteredStream

# Open the stream from an ARFF file
elec_stream = stream_from_file(path_to_csv_or_arff=DATA_PATH+"electricity.arff")
# Create a FilterStream and use the NormalisationFilter
elec_stream_normalised = Stream(CLI=f"-s ({elec_stream.moa_stream.getCLICreationString(elec_stream.moa_stream.__class__)}) \
-f NormalisationFilter ", moa_stream=FilteredStream())

# Creating a learner
ob_learner = OnlineBagging(schema=elec_stream.get_schema(), ensemble_size=5)

# Creating the evaluator
ob_evaluator = ClassificationEvaluator(schema=elec_stream_normalised.get_schema())

while elec_stream_normalised.has_more_instances():
    instance = elec_stream_normalised.next_instance()
    
    prediction = ob_learner.predict(instance)
    ob_evaluator.update(instance.y_label, prediction)
    ob_learner.train(instance)
    # print(instance.x)

ob_evaluator.accuracy()

79.69412076271186